## To download the profile dataset use attached link <br />
https://content.codecademy.com/PRO/paths/data-science/OKCupid-Date-A-Scientist-Starter.zip?_gl=1*1fcshk0*_ga*NzEyNTMwNzI0Ny4xNjU2MzMyNzI3*_ga_3LRZM6TM9L*MTY2MjM5OTY1MS4xMTAuMC4xNjYyMzk5NjUxLjYwLjAuMA..

In [1]:
#import necessary modules for data exploration
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
#load and explore data for understanding
df = pd.read_csv(r'C:\Users\kylew\Codecademy\profiles.csv')

In [3]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   body_type    54650 non-null  object 
 2   diet         35551 non-null  object 
 3   drinks       56961 non-null  object 
 4   drugs        45866 non-null  object 
 5   education    53318 non-null  object 
 6   essay0       54458 non-null  object 
 7   essay1       52374 non-null  object 
 8   essay2       50308 non-null  object 
 9   essay3       48470 non-null  object 
 10  essay4       49409 non-null  object 
 11  essay5       49096 non-null  object 
 12  essay6       46175 non-null  object 
 13  essay7       47495 non-null  object 
 14  essay8       40721 non-null  object 
 15  essay9       47343 non-null  object 
 16  ethnicity    54266 non-null  object 
 17  height       59943 non-null  float64
 18  income       59946 non-null  int64  
 19  job 

In [4]:
#exploration continued
# print(df.head())
def explorer(data):
    ordinals = ['body_type', 'diet', 'drinks', 'drugs', 'education', 'ethnicity', 'job', 'last_online', 'location', 'offspring'\
               ,'orientation', 'pets', 'religion', 'sex', 'sign', 'smokes', 'speaks', 'status']
    for column in data[ordinals].select_dtypes(include='object').columns:
        print(data[column].unique())
explorer(df)

['a little extra' 'average' 'thin' 'athletic' 'fit' nan 'skinny' 'curvy'
 'full figured' 'jacked' 'rather not say' 'used up' 'overweight']
['strictly anything' 'mostly other' 'anything' 'vegetarian' nan
 'mostly anything' 'mostly vegetarian' 'strictly vegan'
 'strictly vegetarian' 'mostly vegan' 'strictly other' 'mostly halal'
 'other' 'vegan' 'mostly kosher' 'strictly halal' 'halal'
 'strictly kosher' 'kosher']
['socially' 'often' 'not at all' 'rarely' nan 'very often' 'desperately']
['never' 'sometimes' nan 'often']
['working on college/university' 'working on space camp'
 'graduated from masters program' 'graduated from college/university'
 'working on two-year college' nan 'graduated from high school'
 'working on masters program' 'graduated from space camp'
 'college/university' 'dropped out of space camp'
 'graduated from ph.d program' 'graduated from law school'
 'working on ph.d program' 'two-year college'
 'graduated from two-year college' 'working on med school'
 'dropped out

In [10]:
#many categories have basically nan values better to be nan
df.replace(to_replace=-1, value=np.nan)
df['body_type'].replace(to_replace='rather not say', value=np.nan)
df['diet'].replace(to_replace='strictly other', value='other')
df['job'].replace(to_replace='rather not say', value=np.nan)
df['status'].replace(to_replace='unknown', value=np.nan)

#function to have location be more general
def fix_location(data):
    new_strings = []
    for string in data['location']:
        new_strings.append(string.split(', ')[1])
    data['location'] = new_strings

#function to remove mistaken characters from offspring strings
def fix_offspring(data):
    new_offspring = []
    for string in data['offspring']:
        if type(string) is str:
            new_offspring.append("'".join(string.split('&rsquo;')))
        else:
            new_offspring.append(np.nan)
    df['offspring'] = new_offspring

#same function as for offspring but for sign
def fix_sign(data):
    new_sign = []
    for string in data['sign']:
        if type(string) is str:
            new_sign.append("'".join(string.split('&rsquo;')))
        else:
            new_sign.append(np.nan)
    df['sign'] = new_sign

#only comment in the data has not yet been cleaned
# fix_location(df)
# fix_offspring(df)
# fix_sign(df)
print(df['location'].unique())
print(df['offspring'].unique())

['california' 'colorado' 'new york' 'oregon' 'arizona' 'hawaii' 'montana'
 'wisconsin' 'virginia' 'spain' 'nevada' 'illinois' 'vietnam' 'ireland'
 'louisiana' 'michigan' 'texas' 'united kingdom' 'massachusetts'
 'north carolina' 'idaho' 'mississippi' 'new jersey' 'florida' 'minnesota'
 'georgia' 'utah' 'washington' 'west virginia' 'connecticut' 'tennessee'
 'rhode island' 'district of columbia' 'british columbia' 'missouri'
 'germany' 'pennsylvania' 'netherlands' 'switzerland' 'mexico' 'ohio']
["doesn't have kids, but might want them" nan "doesn't want kids"
 "doesn't have kids, but wants them" "doesn't have kids" 'wants kids'
 'has a kid' 'has kids' "doesn't have kids, and doesn't want any"
 "has kids, but doesn't want more" "has a kid, but doesn't want more"
 'has a kid, and wants more' 'has kids, and might want more'
 'might want kids' 'has a kid, and might want more'
 'has kids, and wants more']


In [7]:
#three approaches to take, nlp from the essay statements to predict, RF for ordinals as dummies, LR for continous 
#Judging from only binary category is sex this can be used as targets
#0 for male, 1 for female
# labels = np.where(df['sex'] == 'm', 0, 1)

def random_forest(data):
    #import useful modules for random forest learning
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, precision_score
    from sklearn import tree
    
    #clean and seperate data in prep for use
    good_features = ['body_type', 'diet', 'drinks', 'drugs', 'job', 'location', 'offspring', 'orientation', 'smokes', 'status']
    new_data = data.dropna(axis=0, how='any', subset=good_features)
    y = np.where(new_data['sex'] == 'm', 0, 1)
    x = pd.get_dummies(new_data[good_features], drop_first=True)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    #initiate and train RandomForest over range of max_depth
    #self iterated by user over different ranges until best is found
    accuracy_test = []
    for i in range(14, 40):
        rf = RandomForestClassifier(max_depth=i, random_state=42)
        rf.fit(x_train, y_train)
        y_pred = rf.predict(x_test)
        accuracy_test.append(accuracy_score(y_test, y_pred))
    best_depth = 14 + np.argmax(accuracy_test)
    print(f'best depth: {best_depth}')
    
    plt.plot(range(14, 40), accuracy_test)
    plt.xlabel('Max Depth')
    plt.ylabel('Accuracy of Forest')
    plt.show()
    plt.clf()
    
    #refit with best depth and determine feature importance
    rf = RandomForestClassifier(max_depth = best_depth)
    rf.fit(x_train, y_train)
    feat_labels = x_train.columns[0:]
    importance = rf.feature_importances_
    feat_imp = pd.DataFrame(columns=['feature', 'importance'])
    feat_imp['feature'] = feat_labels
    feat_imp['importance'] = importance
    feat_imp.sort_values('importance', axis=0, ascending=False, inplace=True)
    print(feat_imp)
    
    #return predictions of test set can fill with any data that fits the format
    return rf.predict(x_test)
    
# random_forest(df)

In [8]:
#clean the essay data to try and improve NB accuracy
#usage of cleaner does not seem to improve the accuracy of the classifier
def cleaner(data):
    import re
    #remove NaN values
    essays = ['essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']
    new_df = data.dropna(axis=0, how='any', subset=essays)
        
    pattern = r'(\.|\,|\)|\(|<br />)'
    for essay in essays:
        new_strings = []
        temp = ''
        for string in new_df[essay]:
            new_strings.append("".join(re.split(pattern, string)).lower())
        new_df[essay] = new_strings    
    return new_df

#explore the use of naive bayes on essay answers to determine gender
def nb(data):
    #import useful modules for a NB classifier
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.naive_bayes import MultinomialNB
    
    #list of essay labels in df
    essays = ['essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']
    new_df = data.dropna(axis=0, how='any', subset=essays)
    
    #assign target labels and essay data for processing
    #0 for male, 1 for female
    y = np.where(new_df['sex'] == 'm', 0, 1)
    
    #repeat classification for each essay response
    essay_accuracy = []
    for essay in essays:
        
        #initialize counter variable for text
        counter = CountVectorizer()
        counter.fit(new_df[essay])
        #split the data
        x = new_df[essay]
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
        
        #transform data into usable information
        x_train_counts = counter.transform(x_train)
        x_test_counts = counter.transform(x_test)
    
        #init naive bayes classifier
        classifier = MultinomialNB()
        classifier.fit(x_train_counts, y_train)
        essay_accuracy.append(classifier.score(x_test_counts, y_test))
    
    #prints best accuracy and the essay for which that is
    best_accuracy = np.max(essay_accuracy)
    index = np.argmax(essay_accuracy)
    best_essay = essays[index]
    print(f'Best model based on accuracy is {best_essay} with an accuracy of {best_accuracy}')
    
    #plot accuracy for each essay type
    plt.bar(essays, essay_accuracy, tick_label=range(1,10))
    plt.xlabel('essay number')
    plt.ylabel('accuracy score')
    plt.title('Comparison of essay category to accuracy')
    plt.show()
    plt.clf()
    
    #find accuracy if using essay pairs as data
    i = 0
    pair_accuracy = []
    pairs = []
    while i < len(essays)-1:
        n = i+1
        #initialize counter variable for text
        counter = CountVectorizer()
        counter.fit(new_df[essays[i]] + new_df[essays[n]])
        x = new_df[essays[i]] + new_df[essays[n]]
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
        
        #transform data into usable information
        x_train_counts = counter.transform(x_train)
        x_test_counts = counter.transform(x_test)
        
        #init naive bayes classifier
        classifier = MultinomialNB()
        classifier.fit(x_train_counts, y_train)
        pair_accuracy.append(classifier.score(x_test_counts, y_test))
        
        pairs.append(f'{i} & {n}')
        i = i+2
        
    #prints best accuracy and the essay for which that is
    best_pair = np.max(pair_accuracy)
    best_pair_index = np.argmax(pair_accuracy)
    pair_max = pairs[best_pair_index]
    print(f'Best model based on accuracy is {pair_max} with an accuracy of {best_pair}')
    
    #plot accuracy for each essay type
    plt.bar(pairs, pair_accuracy, tick_label=pairs)
    plt.xlabel('essay pairs')
    plt.ylabel('accuracy score')
    plt.title('Comparison of essay category pairs to accuracy')
    plt.show()
    plt.clf()
    
    #find accuracy if used total combined dataset
    counter = CountVectorizer()
    counter.fit(new_df[essays])
    x = new_df['essay1'] + new_df['essay2'] + new_df['essay3'] + new_df['essay4'] + new_df['essay5'] + new_df['essay6'] + new_df['essay7'] + new_df['essay8'] + new_df['essay9']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    #transform data into usable information
    x_train_counts = counter.transform(x_train)
    x_test_counts = counter.transform(x_test)

    #init naive bayes classifier
    classifier = MultinomialNB()
    classifier.fit(x_train_counts, y_train)
    model_accuracy = classifier.score(x_test_counts, y_test)
    print(f'For combined essays model accuracy is {model_accuracy}')
# new_data = cleaner(df)
# nb(df)

In [13]:
#method 3 normalize and use LR on contiinous data then whole data
def normalise(data):
    #import useful mods
    from sklearn.preprocessing import StandardScaler
    
    conts = ['age', 'height', 'income']
    new_data = data.dropna(axis=0, how='any', subset=conts)
    
    scaler = StandardScaler()
    x = scaler.fit_transform(new_data[conts])
    y = new_data['sex']
    return x, y

x, y = normalise(df)

In [15]:
#perform the LR on the prepared data
def linear_mod(x, y):
    #import useful mods
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    
    #seperate data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    lr = LogisticRegression()
    lr.fit(x_train, y_train)
    score = round(lr.score(x_test, y_test), 4)
    print(f'Model has accuracy {score}')

linear_mod(x, y)

Model has accuracy 0.8335


In [45]:
#attempt to feature engineer the categorical dataset for a LR model
def total_lr(data):
    #import useful mods
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import confusion_matrix
    
    #import column lists
    to_drop = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9'\
              ,'last_online']
    ordinals = ['body_type', 'diet', 'drinks', 'drugs', 'education', 'ethnicity', 'job', 'location', 'offspring'\
               ,'orientation', 'pets', 'religion', 'sign', 'smokes', 'speaks', 'status']
    conts = ['age', 'height', 'income']
    
    #clean the data slightly
    new_data = data.drop(to_drop, axis=1)
    new_data.dropna(axis=0, how='any', subset=ordinals+conts, inplace=True)
    
    #OHE the categoric variables
    for column in ordinals:
        holder = pd.get_dummies(new_data[column], prefix=column)
        new_data = pd.concat([new_data, holder], axis=1)
        new_data = new_data.drop(column, axis=1)
    
    #scale the continuous variables
    scaler = StandardScaler()
    new_data[conts] = scaler.fit_transform(new_data[conts])
    
    #seperate data
    y = new_data['sex']
    x = new_data.drop('sex', axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    #perform LR model
    lr = LogisticRegression(max_iter=200)
    lr.fit(x_train, y_train)
    score = lr.score(x_test, y_test)
    print(f'Model accuracy is {score}')
    
    #visualise
    y_pred = lr.predict(x_test)
    print('\nConfusion Matrix for results')
    print(confusion_matrix(y_test, y_pred))
    
total_lr(df)

Model accuracy is 0.8800841514726507

Confusion Matrix for results
[[568 103]
 [ 68 687]]
